## Generar dataset

In [34]:
import numpy as np
import pandas as pd

# Configuración del brazo robótico
l1, l2, l3 = 1, 1, 1  # Longitudes de los segmentos del brazo

# Función para calcular la posición (x, y) del extremo del brazo dado los ángulos
def forward_kinematics(theta1, theta2, theta3):
    x = l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2) + l3 * np.cos(theta1 + theta2 + theta3)
    y = l1 * np.sin(theta1) + l2 * np.sin(theta1 + theta2) + l3 * np.sin(theta1 + theta2 + theta3)
    return x, y

# Generar dataset
num_samples = 1000
theta1 = np.random.uniform(0, 2 * np.pi, num_samples)
theta2 = np.random.uniform(0, 2 * np.pi, num_samples)
theta3 = np.random.uniform(0, 2 * np.pi, num_samples)

data = []
for t1, t2, t3 in zip(theta1, theta2, theta3):
    x, y = forward_kinematics(t1, t2, t3)
    data.append([t1, t2, t3, x, y])

df = pd.DataFrame(data, columns=['theta1', 'theta2', 'theta3', 'x', 'y'])
df.to_csv('robot_arm_dataset.csv', index=False)

## Definir la arquitectura

In [25]:
!pip install tensorflow

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Cargar el dataset
df = pd.read_csv('robot_arm_dataset.csv')

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(df[['theta1', 'theta2', 'theta3']].values)
y = df[['x', 'y']].values

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de la red neuronal con mayor complejidad
model = Sequential([
    Dense(128, input_dim=3, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='linear')
])

# Compilar el modelo con una tasa de aprendizaje menor
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=16)



Epoch 1/200
40/40 [==============================] - 1s 8ms/step - loss: 1.2460 - accuracy: 0.6562 - val_loss: 0.9970 - val_accuracy: 0.7500
Epoch 2/200
40/40 [==============================] - 0s 5ms/step - loss: 0.9512 - accuracy: 0.7641 - val_loss: 0.7852 - val_accuracy: 0.7812
Epoch 3/200
40/40 [==============================] - 0s 5ms/step - loss: 0.7331 - accuracy: 0.7906 - val_loss: 0.5796 - val_accuracy: 0.8188
Epoch 4/200
40/40 [==============================] - 0s 5ms/step - loss: 0.5499 - accuracy: 0.8219 - val_loss: 0.3973 - val_accuracy: 0.8438
Epoch 5/200
40/40 [==============================] - 0s 7ms/step - loss: 0.3954 - accuracy: 0.8672 - val_loss: 0.3233 - val_accuracy: 0.8438
Epoch 6/200
40/40 [==============================] - 0s 5ms/step - loss: 0.3220 - accuracy: 0.8891 - val_loss: 0.2957 - val_accuracy: 0.8375
Epoch 7/200
40/40 [==============================] - 0s 4ms/step - loss: 0.2707 - accuracy: 0.8984 - val_loss: 0.2473 - val_accuracy: 0.8750
Epoch 8/200
4

## Evaluar accuracy

In [37]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

7/7 [==============================] - 0s 4ms/step - loss: 0.0058 - accuracy: 0.9700
Loss: 0.005828638561069965, Accuracy: 0.9700000286102295


## Guardar modelo

In [38]:
# Guardar el modelo si la precisión es adecuada

if accuracy > 0.9:
    model.save('robot_arm_model.h5')
    print("Modelo guardado con éxito")
else:
    print("El modelo no tiene suficiente precisión")

Modelo guardado con éxito


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Descargar modelo

In [39]:
from google.colab import files

# Descargar el archivo .h5
files.download('robot_arm_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>